In [1]:
import numpy as np
import pandas as pd


In [16]:
%matplotlib inline

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv'

above link belongs to mlzoomcamp 

In [3]:
df = pd.read_csv(url)

In [4]:
df_back = df

In [23]:
df = df_back

In [24]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [52]:
selected_columns = ['latitude', 'longitude','minimum_nights', 'number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365' ]

### Question 1

Find a feature with missing values. How many missing values does it have?

### Answer 1

In [26]:
df.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

### Question 2

What's the median (50% percentile) for variable 'minimum_nights'?

### Answer 2

In [27]:
median_minnights = df['minimum_nights'].median()
print(median_minnights)

3.0


### Split the data

<ul>
<li>Shuffle the initial dataset, use seed 42.</li>
<li>Split your data in train/val/test sets, with 60%/20%/20% distribution.</li>
<li>Make sure that the target value ('price') is not in your dataframe.</li>
<li>Apply the log transformation to the price variable using the np.log1p() function.</li>
</ul>

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [30]:
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=42)

In [31]:
y_train = np.log1p(df_train.price.values)
y_val = np.log1p(df_val.price.values)
y_test = np.log1p(df_test.price.values)

In [32]:
del df_train['price']
del df_val['price']
del df_test
['price']

### Question 3

<ul>
<li>We need to deal with missing values for the column from Q1.</li>
<li>We have two options: fill it with 0 or with the mean of this variable.</li>
<li>Try both options. For each, train a linear regression model without regularization using the code from the lessons.</li>
<li>For computing the mean, use the training only! </li>
<li>Use the validation dataset to evaluate the models and compare the RMSE of each option.</li>
<li>Round the RMSE scores to 2 decimal digits using round(score, 2)
Which option gives better RMSE?</li>
</ul>

In [ ]:
df_train = df_train[selected_columns]

In [55]:
df_val = df_val[selected_columns]

In [56]:
df_test = df_test[selected_columns]

In [61]:

def train_linear_regression(X,y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
#I'll explain it with picture
    
    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)

    return w[0], w[1:]

In [87]:
def prepare_X0(df):
    df_num = df[selected_columns]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

In [88]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

### Option 1 

fill with 0

In [60]:
df_train_zero = df_train
df_val_zero = df_val
df_test_zero = df_test    

In [89]:
X_train = prepare_X0(df_train_zero)
w_0, w = train_linear_regression(X_train, y_train)
y_pred = w_0 + X_train.dot(w)
round(rmse(y_train, y_pred),2)

0.65

In [90]:
X_val = prepare_X0(df_val_zero)
y_pred = w_0 + X_val.dot(w)
round(rmse(y_val,y_pred),2)

0.64

In [91]:
X_test = prepare_X0(df_test_zero)
y_pred = w_0 + X_test.dot(w)
round(rmse(y_test,y_pred),2)

0.64

### Option 2

fill with mean val

In [92]:
mean = df_train.reviews_per_month.mean()

In [93]:
def prepare_X(df,fillnav):
    df_num = df[selected_columns]
    df_num = df_num.fillna(fillnav)
    X = df_num.values
    return X

In [98]:
X_train = prepare_X(df_train,0)
w_0, w = train_linear_regression(X_train, y_train)
y_pred = w_0 + X_train.dot(w)
round(rmse(y_train, y_pred),2)

0.65

In [99]:
X_val = prepare_X(df_val,0)
y_pred = w_0 + X_val.dot(w)
round(rmse(y_val,y_pred),2)

0.64

In [100]:
X_test = prepare_X(df_test,0)
y_pred = w_0 + X_test.dot(w)
round(rmse(y_test,y_pred),2)

0.64

### Answer3


Both Are equally good

### Question 4


<ul>
<li>Now let's train a regularized linear regression.</li>
<li>For this question, fill the NAs with 0.</li>
<li>Try different values of r from this list: [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]</li>
<li>Use RMSE to evaluate the model on the validation dataset. </li>
<li>Round the RMSE scores to 2 decimal digits.</li>
<li>Which r gives the best RMSE?</li>
</ul>
If there are multiple opiton, select the smallest r.

In [101]:
def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [102]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    w_0, w = train_linear_regression_reg(X_train, y_train, r=r)
    y_null_reg_val = w_0 + X_val.dot(w)
    rmse_val = np.round(rmse(y_val, y_null_reg_val),2)
    print(r, w_0, rmse_val)

0 -424.995061414696 0.64
1e-06 -424.9378445161687 0.64
0.0001 -419.34862802056364 0.64
0.001 -374.56122541558176 0.64
0.01 -181.11999148803952 0.66
0.1 -29.380106919441364 0.68
1 -3.1316319087220275 0.68
5 -0.6289083682586496 0.68
10 -0.31401805183991943 0.68


### Answer 4 = 0

### Question 5

<ul>
<li>We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score</li>
<li>Try different seed values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].</li>
<li>For each seed, do the train/validation/test split with 60%/20%/20% distribution.</li>
<li>Fill the missing values with 0 and train a model without regularization. </li>
<li>For each seed, evaluate the model on the validation dataset and collect the RMSE scores.</li>
<li>What's the standard deviation of all the scores? To compute the standard deviation, use np.std.</li>
</ul>

Round the result to 3 decimal digits (round(std, 3))


In [103]:
df = df_back

In [109]:
def prepare_X(df):
    df_num = df[selected_columns]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

In [110]:
def train_linear_regression(X,y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)

    return w[0], w[1:]
    

In [111]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [119]:
def splitting(dataFra,randNum):
    df_train_full, df_test = train_test_split(dataFra, test_size=0.2, random_state=randNum)
    df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=randNum)
    y_train = np.log1p(df_train.price.values)
    y_val = np.log1p(df_val.price.values)
    del df_train['price']
    del df_val['price']
    del df_test['price']

    X_train = prepare_X(df_train)
    w_0, w = train_linear_regression(X_train, y_train)
    X_val = prepare_X(df_val)
    y_pred = w_0 + X_val.dot(w)
    

    return round(rmse(y_val,y_pred),3) 


In [124]:
for randNum in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    print (randNum,w_0, splitting(df,randNum))
    

0 -0.31401805183991943 0.641
1 -0.31401805183991943 0.649
2 -0.31401805183991943 0.642
3 -0.31401805183991943 0.646
4 -0.31401805183991943 0.641
5 -0.31401805183991943 0.635
6 -0.31401805183991943 0.648
7 -0.31401805183991943 0.645
8 -0.31401805183991943 0.644
9 -0.31401805183991943 0.645
